**Reference Link:** [RAG Systems Essentials (Analytics Vidhya)](https://courses.analyticsvidhya.com/courses/take/rag-systems-essentials/lessons/60148017-hands-on-deep-dive-into-rag-evaluation-metrics-generator-metrics-i)

# Build a Simple RAG System

## Install OpenAI, and LangChain dependencies

In [1]:
!pip install -qq langchain==0.3.10
!pip install -qq langchain-openai==0.2.12
!pip install -qq langchain-community==0.3.11

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.11 requires langchain<0.4.0,>=0.3.11, but you have langchain 0.3.10 which is incompatible.


## Install Chroma Vector DB and LangChain wrapper

In [2]:
!pip install -qq langchain-chroma==0.1.4

## Install RAG Eval Libraries

In [3]:
!pip install -qq ragas==0.2.8
!pip install -qq deepeval==1.4.7

## Enter Open AI API Key

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

True

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [5]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [6]:
openai_embed_model.embed_query("Hello, world!")

[-0.019143931567668915,
 -0.025292053818702698,
 -0.0017211713129654527,
 0.01883450709283352,
 -0.03382139280438423,
 -0.019682060927152634,
 -0.02102738246321678,
 0.05160655081272125,
 -0.03218010067939758,
 -0.03043118305504322,
 -0.0021508336067199707,
 -0.028924422338604927,
 -0.0024871639907360077,
 -0.03148053586483002,
 0.010291713289916515,
 0.01856544241309166,
 -0.04614454507827759,
 0.04140901193022728,
 0.00043050304520875216,
 0.04116685315966606,
 0.053651440888643265,
 0.0018481360748410225,
 0.004564004950225353,
 0.009955382905900478,
 0.04781274124979973,
 0.002164286794140935,
 -0.00984775647521019,
 0.038422394543886185,
 0.0009131372789852321,
 -0.05209086835384369,
 0.051122233271598816,
 -0.032529886811971664,
 -0.01408552099019289,
 -0.012605667114257812,
 0.013271600939333439,
 0.01856544241309166,
 0.0016320437425747514,
 -0.0008479732787236571,
 -0.012773832306265831,
 -0.029677802696824074,
 -0.004510191734880209,
 -0.015309764072299004,
 0.025668743997812

## Loading and Processing the Data

### Get the dataset

In [0]:
# if you can't download using the following code
# go to https://drive.google.com/file/d/1QkSY9W5RyaBnY8c5FLIsmpPVXoHTQ-fb/view?usp=sharing download it
# manually upload it on colab
# !gdown 1QkSY9W5RyaBnY8c5FLIsmpPVXoHTQ-fb

### Load and Process JSON Documents

In [ ]:
import pandas as pd

df = pd.read_csv('../../docs/rag_eval_docs.csv')
df

In [ ]:
docs = df.to_dict(orient='records')
docs[:3]

In [ ]:
from langchain.docstore.document import Document
processed_docs = []

for doc in docs:
    metadata = {
        "title": doc['title'],
        "id": doc['id'],
    }
    data = doc['context']
    processed_docs.append(Document(page_content=data, metadata=metadata))
processed_docs[:3]

## Index Document Chunks and Embeddings in Vector DB

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [ ]:
# print("Checkpoint 1")

In [ ]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=processed_docs,
                                  collection_name='my_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./my_db")

### Load Vector DB from disk

This is just to show once you have a vector database on disk you can just load and create a connection to it anytime

In [ ]:
# load from disk
chroma_db = Chroma(persist_directory="./my_db",
                   collection_name='my_db',
                   embedding_function=openai_embed_model)

In [ ]:
chroma_db

### Semantic Similarity based Retrieval

We use simple cosine similarity here and retrieve the top 3 similar documents based on the user input query

In [ ]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                              search_kwargs={"k": 3, "score_threshold": 0.3})

In [ ]:
from IPython.display import display, Markdown

def display_docs(docs):
    for doc in docs:
        print('Metadata:', doc.metadata)
        print('Content Brief:')
        display(Markdown(doc.page_content))
        print()

In [ ]:
query = "what is AI?"
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

In [ ]:
query = "how do plants survive?"
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

## Build the RAG Pipeline

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer to the point based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from operator import itemgetter


chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

src_rag_response_chain = (
    {
        "context": (itemgetter('context')
                        |
                    RunnableLambda(format_docs)),
        "question": itemgetter("question")
    }
        |
    rag_prompt_template
        |
    chatgpt
        |
    StrOutputParser()
)

rag_chain_w_sources = (
    {
        "context": similarity_retriever,
        "question": RunnablePassthrough()
    }
        |
    RunnablePassthrough.assign(response=src_rag_response_chain)
)

In [ ]:
query = "What is AI?"
result = rag_chain_w_sources.invoke(query)
result

In [ ]:
query = "How do plants survive?"
result = rag_chain_w_sources.invoke(query)
result